# Impost Packages

In [1]:
import pandas as pd
import numpy as np

# Scrape Test Data and Match the Format with Train

In [2]:
# Read HTML tables
choco_test = pd.read_html('http://flavorsofcacao.com/database_w_REF.html')[0]
choco_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2530 entries, 0 to 2529
Data columns (total 10 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   REF                               2530 non-null   int64  
 1   Company (Manufacturer)            2530 non-null   object 
 2   Company Location                  2530 non-null   object 
 3   Review Date                       2530 non-null   int64  
 4   Country of Bean Origin            2530 non-null   object 
 5   Specific Bean Origin or Bar Name  2530 non-null   object 
 6   Cocoa Percent                     2530 non-null   object 
 7   Ingredients                       2443 non-null   object 
 8   Most Memorable Characteristics    2530 non-null   object 
 9   Rating                            2530 non-null   float64
dtypes: float64(1), int64(2), object(7)
memory usage: 197.8+ KB


In [3]:
choco_test_edit = choco_test.copy()

In [4]:
# Match Column Names with the ones in Train set
choco_test_edit.rename(columns = {"REF": "ref", "Company (Manufacturer)": "company",
                     "Company Location": "company_location", "Review Date": "review_date",
                     "Country of Bean Origin": "country_of_bean_origin", "Specific Bean Origin or Bar Name": "specific_bean_origin_or_bar_name",
                     "Cocoa Percent": "cocoa_percent", "Rating": "rating"}, inplace = True)

To match with the training data set's form, we need to:
1. drop % for cocoa_percent
2. separate Ingredients into counts_of_ingredients, cocoa_butter, vanilla, lecithin, salt, sugar, sweetener_without_sugar (7 new columns)

In [5]:
# drop % sign for cocoa_percent
choco_test_edit['cocoa_percent'] = round(choco_test_edit['cocoa_percent'].str.rstrip('%').astype('float'),1)

In [6]:
choco_test_edit.head()


,ref,company,company_location,review_date,country_of_bean_origin,specific_bean_origin_or_bar_name,cocoa_percent,Ingredients,Most Memorable Characteristics,rating
0,2454,5150,U.S.A.,2019,Tanzania,"Kokoa Kamili, batch 1",76.0,"3- B,S,C","rich cocoa, fatty, bready",3.25
1,2458,5150,U.S.A.,2019,Dominican Republic,"Zorzal, batch 1",76.0,"3- B,S,C","cocoa, vegetal, savory",3.50
2,2454,5150,U.S.A.,2019,Madagascar,"Bejofo Estate, batch 1",76.0,"3- B,S,C","cocoa, blackberry, full body",3.75
3,2542,5150,U.S.A.,2021,Fiji,"Matasawalevu, batch 1",68.0,"3- B,S,C","chewy, off, rubbery",3.00
4,2546,5150,U.S.A.,2021,Venezuela,"Sur del Lago, batch 1",72.0,"3- B,S,C","fatty, earthy, moss, nutty,chalky",3.00


In [7]:
# separate 'Ingredients' column into 7 new columns to be consistent with the training set,
# i.e., counts_of_ingredients, cocoa_butter, vanilla, lecithin, salt, sugar, sweetener_without_sugar

# create new columns
counts_of_ingredients = [] # the first number; this should be a list of integers
cocoa_butter = [] # C -> have_cocoa_butter; have_not_cocoa_butter
vanilla = [] # V -> have_vanilla; have_not_vanilla
lecithin = [] # L -> have_lecithin; have_not_lecithin
salt = [] # Sa -> have_salt; have_not_salt
sugar = [] # S -> have_sugar; have_not_sugar
sweetener_without_sugar = [] # S* -> have_sweetener_without_sugar; have_not_sweetener_without_sugar
delete = []

for ind, row in choco_test_edit.iterrows():
    cell = row['Ingredients']
    if type(cell) == str:
        l = cell.replace('- ', '-')
        l = l.replace('-', ',').split(',') # this splits each input into a list of strings

        # append the first number to the list of counts_of_ingredients
        counts_of_ingredients.append(int(l[0]))

        # cocoa_butter
        if 'C' in l[1:]:
            cocoa_butter.append('have_cocoa_butter')
        else:
            cocoa_butter.append('have_not_cocoa_butter')

        # vanilla
        if 'V' in l[1:]:
            vanilla.append('have_vanilla')
        else:
            vanilla.append('have_not_vanilla')

        # lecithin
        if 'L' in l[1:]:
            lecithin.append('have_lecithin')
        else:
            lecithin.append('have_not_lecithin')

        # salt
        if 'Sa' in l[1:]:
            salt.append('have_salt')
        else:
            salt.append('have_not_salt')

        # sugar
        if 'S' in l[1:]:
            sugar.append('have_sugar')
        else:
            sugar.append('have_not_sugar')

        # sweetener_without_sugar
        if 'S*' in l[1:]:
            sweetener_without_sugar.append('have_sweetener_without_sugar')
        else:
            sweetener_without_sugar.append('have_not_sweetener_without_sugar')
    else:
        # if no info for this chocolate, record it's index and delete it from the data set later
        delete.append(ind)

In [8]:
# Drop the data with no ingredients info provided

choco_test_edit.drop(delete, inplace = True)

In [9]:
# Add these as new columns to dataframe
choco_test_edit['counts_of_ingredients'] = counts_of_ingredients
choco_test_edit['cocoa_butter'] = cocoa_butter
choco_test_edit['vanilla'] = vanilla
choco_test_edit['lecithin'] = lecithin
choco_test_edit['salt'] = salt
choco_test_edit['sugar'] = sugar
choco_test_edit['sweetener_without_sugar'] = sweetener_without_sugar

In [10]:
# drop the column Ingredients
choco_test_edit.drop(columns='Ingredients', inplace=True)

In [11]:
# rename Most Memorable Characteristics as taste
choco_test_edit.rename(columns={"Most Memorable Characteristics": "taste"}, inplace=True)

In [12]:
choco_test_edit1 = choco_test_edit.copy()

In [13]:
# Regularize the formatting for the 'taste' column
new = []
for index, row in choco_test_edit1.iterrows():
    l1 = row['taste'].replace(', ', ',')
    l2 = l1.replace(',', ', ')
    new.append(l2)
    
choco_test_edit1['taste'] = new

In [14]:
# Reset the index
choco_test_edit1.index = np.arange(len(choco_test_edit1))

In [15]:
# Match country names - Change U.S.A. to U.S.A and U.K. to U.k.
choco_test_edit1.replace('U.S.A.', 'U.S.A', inplace = True)

choco_test_edit1.replace('U.K.', 'U.k.', inplace = True)

In [16]:
choco_test_edit1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2443 entries, 0 to 2442
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ref                               2443 non-null   int64  
 1   company                           2443 non-null   object 
 2   company_location                  2443 non-null   object 
 3   review_date                       2443 non-null   int64  
 4   country_of_bean_origin            2443 non-null   object 
 5   specific_bean_origin_or_bar_name  2443 non-null   object 
 6   cocoa_percent                     2443 non-null   float64
 7   taste                             2443 non-null   object 
 8   rating                            2443 non-null   float64
 9   counts_of_ingredients             2443 non-null   int64  
 10  cocoa_butter                      2443 non-null   object 
 11  vanilla                           2443 non-null   object 
 12  lecith

In [17]:
choco_test_edit1

,ref,company,company_location,review_date,country_of_bean_origin,specific_bean_origin_or_bar_name,cocoa_percent,taste,rating,counts_of_ingredients,cocoa_butter,vanilla,lecithin,salt,sugar,sweetener_without_sugar
0,2454,5150,U.S.A,2019,Tanzania,"Kokoa Kamili, batch 1",76.0,"rich cocoa, fatty, bready",3.25,3,have_cocoa_butter,have_not_vanilla,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar
1,2458,5150,U.S.A,2019,Dominican Republic,"Zorzal, batch 1",76.0,"cocoa, vegetal, savory",3.50,3,have_cocoa_butter,have_not_vanilla,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar
2,2454,5150,U.S.A,2019,Madagascar,"Bejofo Estate, batch 1",76.0,"cocoa, blackberry, full body",3.75,3,have_cocoa_butter,have_not_vanilla,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar
3,2542,5150,U.S.A,2021,Fiji,"Matasawalevu, batch 1",68.0,"chewy, off, rubbery",3.00,3,have_cocoa_butter,have_not_vanilla,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar
4,2546,5150,U.S.A,2021,Venezuela,"Sur del Lago, batch 1",72.0,"fatty, earthy, moss, nutty, chalky",3.00,3,have_cocoa_butter,have_not_vanilla,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2438,1205,Zotter,Austria,2014,Blend,Raw,80.0,"waxy, cloying, vegetal",2.75,4,have_cocoa_butter,have_not_vanilla,have_not_lecithin,have_salt,have_not_sugar,have_sweetener_without_sugar
2439,1996,Zotter,Austria,2017,Colombia,"APROCAFA, Acandi",75.0,"strong nutty, marshmallow",3.75,3,have_cocoa_butter,have_not_vanilla,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar
2440,2036,Zotter,Austria,2018,Blend,"Dry Aged, 30 yr Anniversary bar",75.0,"fatty, earthy, cocoa",3.00,3,have_cocoa_butter,have_not_vanilla,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar
2441,2170,Zotter,Austria,2018,Congo,Mountains of the Moon,70.0,"fatty, mild nuts, mild fruit",3.25,3,have_cocoa_butter,have_not_vanilla,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar


# Clean and One-hot Encode Train and Test Data

## Clean Train Data

In [18]:
# Import data
choco_train = pd.read_csv("data/chocolate.csv", index_col=0)

In [19]:
pd.set_option('display.max_columns', None)
choco_train.head()

,ref,company,company_location,review_date,country_of_bean_origin,specific_bean_origin_or_bar_name,cocoa_percent,rating,counts_of_ingredients,beans,cocoa_butter,vanilla,lecithin,salt,sugar,sweetener_without_sugar,first_taste,second_taste,third_taste,fourth_taste
0,2454,5150,U.S.A,2019,Madagascar,"Bejofo Estate, batch 1",76.0,3.75,3,have_bean,have_cocoa_butter,have_not_vanila,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar,cocoa,blackberry,full body,NaN
1,2458,5150,U.S.A,2019,Dominican republic,"Zorzal, batch 1",76.0,3.50,3,have_bean,have_cocoa_butter,have_not_vanila,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar,cocoa,vegetal,savory,NaN
2,2454,5150,U.S.A,2019,Tanzania,"Kokoa Kamili, batch 1",76.0,3.25,3,have_bean,have_cocoa_butter,have_not_vanila,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar,rich cocoa,fatty,bready,NaN
3,797,A. Morin,France,2012,Peru,Peru,63.0,3.75,4,have_bean,have_cocoa_butter,have_not_vanila,have_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar,fruity,melon,roasty,NaN
4,797,A. Morin,France,2012,Bolivia,Bolivia,70.0,3.50,4,have_bean,have_cocoa_butter,have_not_vanila,have_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar,vegetal,nutty,NaN,NaN


In [20]:
choco_train_edit = choco_train.copy()

In [21]:
# Combined the first_taste, second_taste, third_taste and fourth_taste columns into a new column called "taste"
cols = ['first_taste','second_taste','third_taste','fourth_taste']

new_col = []

for i in range(len(choco_train_edit)):
    s = ''
    for col in cols:
        if isinstance(choco_train_edit[col][i], str):
            if not s:
                s = choco_train_edit[col][i]
            else:
                s_new = ', ' + choco_train_edit[col][i]
                s += s_new
    new_col.append(s)
    
choco_train_edit['taste'] = new_col

In [22]:
# Deleted the unuseful columns: 
# 'beans' - all have beans contained
# - 'first_taste', 'second_taste', 'third_taste', 'fourth_taste' 
#     - the test set does not have these distingished, but just one list of strings

choco_train_edit.drop(columns=['beans','first_taste','second_taste','third_taste','fourth_taste'], inplace=True)

choco_train_edit.head()

,ref,company,company_location,review_date,country_of_bean_origin,specific_bean_origin_or_bar_name,cocoa_percent,rating,counts_of_ingredients,cocoa_butter,vanilla,lecithin,salt,sugar,sweetener_without_sugar,taste
0,2454,5150,U.S.A,2019,Madagascar,"Bejofo Estate, batch 1",76.0,3.75,3,have_cocoa_butter,have_not_vanila,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar,"cocoa, blackberry, full body"
1,2458,5150,U.S.A,2019,Dominican republic,"Zorzal, batch 1",76.0,3.50,3,have_cocoa_butter,have_not_vanila,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar,"cocoa, vegetal, savory"
2,2454,5150,U.S.A,2019,Tanzania,"Kokoa Kamili, batch 1",76.0,3.25,3,have_cocoa_butter,have_not_vanila,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar,"rich cocoa, fatty, bready"
3,797,A. Morin,France,2012,Peru,Peru,63.0,3.75,4,have_cocoa_butter,have_not_vanila,have_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar,"fruity, melon, roasty"
4,797,A. Morin,France,2012,Bolivia,Bolivia,70.0,3.50,4,have_cocoa_butter,have_not_vanila,have_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar,"vegetal, nutty"


In [23]:
choco_train_edit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2224 entries, 0 to 2223
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ref                               2224 non-null   int64  
 1   company                           2224 non-null   object 
 2   company_location                  2224 non-null   object 
 3   review_date                       2224 non-null   int64  
 4   country_of_bean_origin            2224 non-null   object 
 5   specific_bean_origin_or_bar_name  2224 non-null   object 
 6   cocoa_percent                     2224 non-null   float64
 7   rating                            2224 non-null   float64
 8   counts_of_ingredients             2224 non-null   int64  
 9   cocoa_butter                      2224 non-null   object 
 10  vanilla                           2224 non-null   object 
 11  lecithin                          2224 non-null   object 
 12  salt  

## One-hot Encoding

We first combine train and test data to do one-hot encoding for all and later separate them.

According to the data's website(http://flavorsofcacao.com/chocolate_database.html), the reviews are sorted by ref with 
the highest number being the newest. So we can split based on ref.

In [24]:
print('The max reference number in the training set is ', max(choco_train_edit['ref']))

The max reference number in the training set is  2490


In [25]:
# Check for ref == 2490
choco_train_edit.loc[choco_train_edit['ref'] == 2490]

,ref,company,company_location,review_date,country_of_bean_origin,specific_bean_origin_or_bar_name,cocoa_percent,rating,counts_of_ingredients,cocoa_butter,vanilla,lecithin,salt,sugar,sweetener_without_sugar,taste
1453,2490,MUCHO,Mexico,2020,Mexico,"Finca La Rioja E., Cacahoatan, Chiapas",70.0,3.00,3,have_cocoa_butter,have_not_vanila,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar,"light color, spicy, sweet, sour"
2078,2490,Tribe,Canada,2020,Vietnam,"Dak Lak, batch 0A08",70.0,2.75,2,have_not_cocoa_butter,have_not_vanila,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar,"tart, sweet, sour, savory"
2079,2490,Tribe,Canada,2020,Honduras,"Copan, batch 0A03",70.0,2.75,2,have_not_cocoa_butter,have_not_vanila,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar,"woody, sweet, unrefined"


In [26]:
choco_test_edit1.loc[choco_test_edit1['ref'] == 2490]

,ref,company,company_location,review_date,country_of_bean_origin,specific_bean_origin_or_bar_name,cocoa_percent,taste,rating,counts_of_ingredients,cocoa_butter,vanilla,lecithin,salt,sugar,sweetener_without_sugar
1592,2490,MUCHO,Mexico,2020,Mexico,"Finca La Rioja E., Cacahoatan, Chiapas",70.0,"light color, spicy, sweet, sour",3.00,3,have_cocoa_butter,have_not_vanilla,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar
1660,2490,Odyssey,U.S.A,2020,Guatemala,Cahabon,70.0,"damp woody, leather, cocoa",3.00,2,have_not_cocoa_butter,have_not_vanilla,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar
2279,2490,Tribe,Canada,2020,Vietnam,"Dak Lak, batch 0A08",70.0,"tart, sweet, sour, savory",2.75,2,have_not_cocoa_butter,have_not_vanilla,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar
2280,2490,Tribe,Canada,2020,Honduras,"Copan, batch 0A03",70.0,"woody, sweet, unrefined",2.75,2,have_not_cocoa_butter,have_not_vanilla,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar


Index 1660 is not in training set, so we add it back after deleting it

In [27]:
# Use the newer reviews as test data
newdf = choco_test_edit1.loc[choco_test_edit1['ref'] > 2490].copy()

choco_test_edit2 = newdf.append(choco_test_edit1.iloc[1660])

/var/folders/h8/fpbfg81d2t55k9x_v30zfp9c0000gn/T/ipykernel_10973/354942719.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  choco_test_edit2 = newdf.append(choco_test_edit1.iloc[1660])


In [28]:
choco_test_edit2.index = np.arange(len(choco_test_edit2))

In [29]:
choco_test_edit2

,ref,company,company_location,review_date,country_of_bean_origin,specific_bean_origin_or_bar_name,cocoa_percent,taste,rating,counts_of_ingredients,cocoa_butter,vanilla,lecithin,salt,sugar,sweetener_without_sugar
0,2542,5150,U.S.A,2021,Fiji,"Matasawalevu, batch 1",68.0,"chewy, off, rubbery",3.00,3,have_cocoa_butter,have_not_vanilla,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar
1,2546,5150,U.S.A,2021,Venezuela,"Sur del Lago, batch 1",72.0,"fatty, earthy, moss, nutty, chalky",3.00,3,have_cocoa_butter,have_not_vanilla,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar
2,2546,5150,U.S.A,2021,Uganda,"Semuliki Forest, batch 1",80.0,"mildly bitter, basic cocoa, fatty",3.25,3,have_cocoa_butter,have_not_vanilla,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar
3,2542,5150,U.S.A,2021,India,"Anamalai, batch 1",68.0,"milk brownie, macadamia, chewy",3.50,3,have_cocoa_butter,have_not_vanilla,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar
4,2648,A. Morin,France,2021,Mexico,La Joya,70.0,"light color, fruit, yogurt",4.00,4,have_cocoa_butter,have_not_vanilla,have_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,2618,Zac Squared,U.S.A,2021,Trinidad,Rio Claro,70.0,"nutty, fruity, sweet",3.25,3,have_cocoa_butter,have_not_vanilla,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar
219,2618,Zac Squared,U.S.A,2021,Vietnam,Ben Tre,70.0,"tart, cherry, cocoa, mild sour",3.50,3,have_cocoa_butter,have_not_vanilla,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar
220,2618,Zac Squared,U.S.A,2021,Dominican Republic,Zorzal,70.0,"sweet, rich, cocoa, fruity",3.50,3,have_cocoa_butter,have_not_vanilla,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar
221,2700,Zacharias,U.S.A,2021,Bolivia,"Tranquilidad, wild Bolivia",70.0,"sweet, tart, sour",3.00,3,have_cocoa_butter,have_not_vanilla,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar


Now we combine the test and train to do encoding for all

In [30]:
choco_combine = pd.concat([choco_train_edit,choco_test_edit2]).reset_index(drop=True)

In [31]:
choco_combine

,ref,company,company_location,review_date,country_of_bean_origin,specific_bean_origin_or_bar_name,cocoa_percent,rating,counts_of_ingredients,cocoa_butter,vanilla,lecithin,salt,sugar,sweetener_without_sugar,taste
0,2454,5150,U.S.A,2019,Madagascar,"Bejofo Estate, batch 1",76.0,3.75,3,have_cocoa_butter,have_not_vanila,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar,"cocoa, blackberry, full body"
1,2458,5150,U.S.A,2019,Dominican republic,"Zorzal, batch 1",76.0,3.50,3,have_cocoa_butter,have_not_vanila,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar,"cocoa, vegetal, savory"
2,2454,5150,U.S.A,2019,Tanzania,"Kokoa Kamili, batch 1",76.0,3.25,3,have_cocoa_butter,have_not_vanila,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar,"rich cocoa, fatty, bready"
3,797,A. Morin,France,2012,Peru,Peru,63.0,3.75,4,have_cocoa_butter,have_not_vanila,have_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar,"fruity, melon, roasty"
4,797,A. Morin,France,2012,Bolivia,Bolivia,70.0,3.50,4,have_cocoa_butter,have_not_vanila,have_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar,"vegetal, nutty"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2442,2618,Zac Squared,U.S.A,2021,Trinidad,Rio Claro,70.0,3.25,3,have_cocoa_butter,have_not_vanilla,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar,"nutty, fruity, sweet"
2443,2618,Zac Squared,U.S.A,2021,Vietnam,Ben Tre,70.0,3.50,3,have_cocoa_butter,have_not_vanilla,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar,"tart, cherry, cocoa, mild sour"
2444,2618,Zac Squared,U.S.A,2021,Dominican Republic,Zorzal,70.0,3.50,3,have_cocoa_butter,have_not_vanilla,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar,"sweet, rich, cocoa, fruity"
2445,2700,Zacharias,U.S.A,2021,Bolivia,"Tranquilidad, wild Bolivia",70.0,3.00,3,have_cocoa_butter,have_not_vanilla,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar,"sweet, tart, sour"


For later use, note that the test set start from index 2224 (to the end)

In [32]:
choco_combine.iloc[2224]

ref                                                             2542
company                                                         5150
company_location                                               U.S.A
review_date                                                     2021
country_of_bean_origin                                          Fiji
specific_bean_origin_or_bar_name               Matasawalevu, batch 1
cocoa_percent                                                   68.0
rating                                                           3.0
counts_of_ingredients                                              3
cocoa_butter                                       have_cocoa_butter
vanilla                                             have_not_vanilla
lecithin                                           have_not_lecithin
salt                                                   have_not_salt
sugar                                                     have_sugar
sweetener_without_sugar           

### One-hot encoding the 6 ingredient columns

Namely 'cocoa_butter', 'vanilla', 'lecithin', 'salt', 'sugar', 'sweetener_without_sugar'

In [33]:
# Make the categorical variables into 0s and 1s

choco_combine = pd.get_dummies(choco_combine, columns=['cocoa_butter', 'vanilla', 'lecithin', 
                                                 'salt', 'sugar', 'sweetener_without_sugar'], drop_first=True)


In [34]:
choco_combine.columns

Index(['ref', 'company', 'company_location', 'review_date',
       'country_of_bean_origin', 'specific_bean_origin_or_bar_name',
       'cocoa_percent', 'rating', 'counts_of_ingredients', 'taste',
       'cocoa_butter_have_not_cocoa_butter', 'vanilla_have_not_vanilla',
       'vanilla_have_vanila', 'vanilla_have_vanilla',
       'lecithin_have_not_lecithin', 'salt_have_salt', 'sugar_have_sugar',
       'sweetener_without_sugar_have_sweetener_without_sugar'],
      dtype='object')

In [35]:
# Rename the new columns

choco_combine.rename(columns = {"cocoa_butter_have_not_cocoa_butter": "cocoa_butter", 
                            "vanilla_have_vanila": "vanilla",
                            "lecithin_have_not_lecithin": "lecithin",
                             "salt_have_salt": "salt",
                            "sugar_have_sugar": "sugar",
                            "sweetener_without_sugar_have_sweetener_without_sugar": "sweetener_without_sugar",}, 
                            inplace = True)

pd.get_dummies() made have_not_cocoa_butter and have_not_lecithin as 1s, so we switch 0 and 1 in those two columns

In [36]:
choco_combine['cocoa_butter'] = - choco_combine['cocoa_butter'] + 1
choco_combine['lecithin'] = - choco_combine['lecithin'] + 1

### One-hot encoding 18 most frequent tastes

We identified 18 most frequently appeared tastes in the 'taste' column and sorted out the synonyms to these words based on the Review Guide from the data webpage (http://flavorsofcacao.com/review_guide.html) and human judgement.

Frequency in order: 'nutty', 'sweet', 'cocoa', 'roasty', 'creamy', 'earthy', 'sandy', 'fatty', 'floral', 'intense', 'spicy', 'sour', 'molasses', 'woody', 'vanilla_taste', 'sticky', 'fruit', 'smokey'

In [37]:
choco_combine1 = choco_combine.copy()

In [38]:
# The 18 taste groups we identified, written as strings

nutty_st = 'nutty, nut, nuts, hazelnut, macadamia, peanut, almond, walnut, roasted nuts, mild nutty, burnt nuts, pistachio'
sweet_st = 'sweet, marshmallow, honey, caramel, butterscotch, candy-like, candy'
cocoa_st = 'cocoa, basic cocoa, brownie, fudge, hot chocolate, chocolate, rich cocoa, mild cocoa, deep cocoa, cocoa base, intense cocoa'
roasty_st = 'roasty, roasted nuts, burnt, tobacco, coffee, heavy roast, high roast, burnt nuts, dark roast, mild roasty'
creamy_st = 'creamy, peanut butter, butterscotch, buttery, cream'
earthy_st = 'earthy, muddy, hay, dirt, intense earthy'
sandy_st = 'sandy, chalky, powdery, ashey, grits, large grits, grainy, grit, very sandy, gritty'
fatty_st = 'fatty, oily'
floral_st = 'floral, perfumes, perfume'
intense_st = 'intense, rich, strong, bold, intense cocoa'
spicy_st = 'spicy, spice, black pepper, mint, pungent, pepper, minty, spciy, spicy pepper, spices, strong spice, hot spice, cinamon, herbs, ginger'
sour_st = 'sour, acidic, sour milk, high acidity, mild sour, vinegar, sour banana, sour orange, sour notes, late sour, sour fruit, off sour'
molasses_st = 'molasses, molassses'
woody_st = 'woody, woodsy, burnt wood, wood'
vanilla_st = 'vanilla, strong vanilla, vanilla domintates' 
sticky_st = 'sticky, chewy'
fruit_st = 'fruit, fruity, citrus, grapes, fig, raspberry, strawberry, raisins, coconut, cherry, banana, blueberry, citrus, grapes, bright fruit, red fruit, dark berry, lemon, berry, orange, melon, red berry, mild fruit, raisin, blackberry, berries, oranges, apple, sour orange, peach, sour banana, peach, pear, papaya, grapefruit, dates, dried fruit' 
smokey_st = 'smokey, smoke, mild smoke, intensely smokey, intense smoke, harsh smoke'


In [39]:
# Make the strings to lists of strings
nutty_ls = nutty_st.split(', ')
sweet_ls = sweet_st.split(', ')
cocoa_ls = cocoa_st.split(', ')
roasty_ls = roasty_st.split(', ')
creamy_ls = creamy_st.split(', ')
earthy_ls = earthy_st.split(', ')
sandy_ls = sandy_st.split(', ')
fatty_ls = fatty_st.split(', ')
floral_ls = floral_st.split(', ')
intense_ls = intense_st.split(', ')
spicy_ls = spicy_st.split(', ')
sour_ls = sour_st.split(', ')
molasses_ls = molasses_st.split(', ')
woody_ls = woody_st.split(', ')
vanilla_ls = vanilla_st.split(', ')
sticky_ls = sticky_st.split(', ')
fruit_ls = fruit_st.split(', ')
smokey_ls = smokey_st.split(', ')


In [40]:
# For simplicity, make the 18 lists into a list of lists

tastes_ls_ls = [nutty_ls, sweet_ls, cocoa_ls, roasty_ls, creamy_ls, earthy_ls, sandy_ls, fatty_ls, floral_ls, 
                intense_ls, spicy_ls, sour_ls, molasses_ls, woody_ls, vanilla_ls, sticky_ls, fruit_ls, smokey_ls]



In [41]:
# Make 18 * len(choc_test) matrix, each row corresponds to a new column to be added later to the df, 
# initially set all to 0s
# Each row correponds to (from row 0 to row 17):
# nutty, sweet, cocoa, roasty, creamy, earthy, sandy, fatty, floral, intense, spicy, sour, molasses, 
# woody, vanilla, sticky, fruit, smokey

tastes_mat = np.zeros((18, len(choco_combine1))).astype(int)


In [42]:
# Go through each row's 'taste' and record based on our list to tastes_mat

samp_ind = 0 # this corresponds to the row index of the df

for tastes in choco_combine1['taste']:
    
    # tastes is a string, make it to a list of strings
    tastes_ls = tastes.split(', ')
    
    for t in tastes_ls:
        
        # go through each one of the 18 tastes
        for i in range(18):
            
            # check if t belongs to the ith taste, if so, change the responding record from 0 to 1
            if t in tastes_ls_ls[i]:
                tastes_mat[i,samp_ind] = 1
                
    samp_ind += 1
                

In [43]:
# Create 18 new columns
new_col_names = ['nutty', 'sweet', 'cocoa', 'roasty', 'creamy', 'earthy', 'sandy', 'fatty', 'floral', 'intense', 
                    'spicy', 'sour', 'molasses', 'woody', 'vanilla_taste', 'sticky', 'fruit', 'smokey']

### Note: name 'vanilla_taste' since already have ''vanilla' column as an ingredient

ind = 0
for name in new_col_names:
    choco_combine1[name] = tastes_mat[ind,:]
    ind += 1

In [44]:
choco_combine1

,ref,company,company_location,review_date,country_of_bean_origin,specific_bean_origin_or_bar_name,cocoa_percent,rating,counts_of_ingredients,taste,cocoa_butter,vanilla_have_not_vanilla,vanilla,vanilla_have_vanilla,lecithin,salt,sugar,sweetener_without_sugar,nutty,sweet,cocoa,roasty,creamy,earthy,sandy,fatty,floral,intense,spicy,sour,molasses,woody,vanilla_taste,sticky,fruit,smokey
0,2454,5150,U.S.A,2019,Madagascar,"Bejofo Estate, batch 1",76.0,3.75,3,"cocoa, blackberry, full body",1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,2458,5150,U.S.A,2019,Dominican republic,"Zorzal, batch 1",76.0,3.50,3,"cocoa, vegetal, savory",1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2454,5150,U.S.A,2019,Tanzania,"Kokoa Kamili, batch 1",76.0,3.25,3,"rich cocoa, fatty, bready",1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,797,A. Morin,France,2012,Peru,Peru,63.0,3.75,4,"fruity, melon, roasty",1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,797,A. Morin,France,2012,Bolivia,Bolivia,70.0,3.50,4,"vegetal, nutty",1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2442,2618,Zac Squared,U.S.A,2021,Trinidad,Rio Claro,70.0,3.25,3,"nutty, fruity, sweet",1,1,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2443,2618,Zac Squared,U.S.A,2021,Vietnam,Ben Tre,70.0,3.50,3,"tart, cherry, cocoa, mild sour",1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2444,2618,Zac Squared,U.S.A,2021,Dominican Republic,Zorzal,70.0,3.50,3,"sweet, rich, cocoa, fruity",1,1,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
2445,2700,Zacharias,U.S.A,2021,Bolivia,"Tranquilidad, wild Bolivia",70.0,3.00,3,"sweet, tart, sour",1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


## Create Continent Columns

'company_location' and 'country_of_bean_origin' contains a large number of countries. If just using as countries as a feature, it may end up with having too many categories and so not revealing much interesting information. So we instead map these countries to their continents and create related columns to the dataframe.

In [45]:
choco_combine1.columns

Index(['ref', 'company', 'company_location', 'review_date',
       'country_of_bean_origin', 'specific_bean_origin_or_bar_name',
       'cocoa_percent', 'rating', 'counts_of_ingredients', 'taste',
       'cocoa_butter', 'vanilla_have_not_vanilla', 'vanilla',
       'vanilla_have_vanilla', 'lecithin', 'salt', 'sugar',
       'sweetener_without_sugar', 'nutty', 'sweet', 'cocoa', 'roasty',
       'creamy', 'earthy', 'sandy', 'fatty', 'floral', 'intense', 'spicy',
       'sour', 'molasses', 'woody', 'vanilla_taste', 'sticky', 'fruit',
       'smokey'],
      dtype='object')

In [46]:
from scipy.stats import f_oneway
import pycountry
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2

In [47]:
# First change the names so that we can use pycountry_convert package
d = {'U.S.A':'United States', 'U.s.a.':'United States', 'U.k.':'United Kingdom', 
     'Wales':'United Kingdom', 'Russia':'Russian Federation', 'Scotland':'United Kingdom',  
     'St. lucia':'Saint Lucia', 'Sao tome & principe':'Sao Tome and Principe',
     'Sao tome':'Sao Tome and Principe', 'Principe':'Sao Tome and Principe',
     'St.vincent-grenadines':'Saint Vincent and the Grenadines', 
     'U.a.e.':'United Arab Emirates', 'Ivory coast':"Republic of Côte d'Ivoire",
     'Burma':'Myanmar', 'Trinidad':'Trinidad and Tobago', 'Bolvia':'Bolivia',
     'Tobago':'Trinidad and Tobago', 'Sumatra':'Indonesia', 'Sulawesi':'Indonesia',
     'Dr congo':'Congo'}

### company_continent for company_location

In [48]:
for k, v in d.items():
    choco_combine1['company_location'] = choco_combine1['company_location'].replace(k, v)

In [49]:
def get_continent(country):
    
    try:
        cn_continent = country_alpha2_to_continent_code(pycountry.countries.lookup(country.lower()).alpha_2)
    except:
        cn_continent = 'Unknown' 
    return cn_continent

In [50]:
continent = []
for i in range(len(choco_combine1)):
    continent.append(get_continent(choco_combine1.iloc[i]['company_location']))

In [51]:
choco_combine1['company_continent'] = continent

### origin_continent for country_of_bean_origin

In [52]:
for k, v in d.items():
    choco_combine1['country_of_bean_origin'] = choco_combine1['country_of_bean_origin'].replace(k, v)

In [53]:
origin_continent = []
for i in range(len(choco_combine1)):
    origin_continent.append(get_continent(choco_combine1.iloc[i]['country_of_bean_origin']))

In [54]:
choco_combine1['origin_continent'] = origin_continent

In [55]:
choco_combine1['origin_continent'].unique()

array(['AF', 'NA', 'SA', 'OC', 'AS', 'Unknown'], dtype=object)

## Target Encoding

### For continent columns

In [56]:
choco_combine2 = choco_combine1.copy()

#### 1. Encode the bean origin continent

##### 1.1 Decide on whether an __additive smoothing__ is needed

Check the count of each bean origin, the mean and std of each origin. 

In [57]:
Count_BO = choco_combine2.groupby('origin_continent')['rating'].count()

In [58]:
Count_BO

origin_continent
AF         351
AS         171
NA         767
OC          94
SA         917
Unknown    147
Name: rating, dtype: int64

In [59]:
type(Count_BO)

pandas.core.series.Series

In [60]:
Mu_BO = choco_combine2.groupby('origin_continent')['rating'].mean()
Sigma_BO = choco_combine2.groupby('origin_continent')['rating'].std()

In [61]:
Mu_BO

origin_continent
AF         3.218661
AS         3.203216
NA         3.211669
OC         3.223404
SA         3.224100
Unknown    3.086735
Name: rating, dtype: float64

In [62]:
Sigma_BO

origin_continent
AF         0.409809
AS         0.348321
NA         0.394740
OC         0.387069
SA         0.448734
Unknown    0.576103
Name: rating, dtype: float64

In [63]:
Sigma_BO.values/Sigma_BO.values

array([1., 1., 1., 1., 1., 1.])

In [64]:
df_BO_summary = pd.DataFrame({'Bean origin': Count_BO.index, 
                              'Count': Count_BO.values,
                              'Mean of ratings': Mu_BO.values,
                              'Std of ratings': Sigma_BO.values,
                              'c.o.v': Sigma_BO.values/Mu_BO.values})
# One way to interpret the relative magnitude of the standard deviation is to divide it by the mean. 
# This is called the coefficient of variation

In [65]:
df_BO_summary

,Bean origin,Count,Mean of ratings,Std of ratings,c.o.v
0,AF,351,3.218661,0.409809,0.127323
1,AS,171,3.203216,0.348321,0.108741
2,NA,767,3.211669,0.394740,0.122908
3,OC,94,3.223404,0.387069,0.120081
4,SA,917,3.224100,0.448734,0.139181
5,Unknown,147,3.086735,0.576103,0.186638


Go ahead without smoothing first

In [66]:
# alternative coding
stats_BO = choco_combine2['rating'].groupby(choco_combine2['origin_continent']).agg(['count', 'mean', 'std'])

In [67]:
stats_BO

,count,mean,std
origin_continent,,,
AF,351,3.218661,0.409809
AS,171,3.203216,0.348321
NA,767,3.211669,0.394740
OC,94,3.223404,0.387069
SA,917,3.224100,0.448734
Unknown,147,3.086735,0.576103


##### 1.2 Encode

In [68]:
choco_combine2['origin_continent'] = choco_combine2['origin_continent'].map(Mu_BO)

#### 2. Encode the company continent

In [69]:
stats_CC = choco_combine2['rating'].groupby(choco_combine2['company_continent']).agg(['count', 'mean', 'std'])
stats_CC

,count,mean,std
company_continent,,,
AF,25,3.050000,0.462106
AS,103,3.203883,0.351376
EU,704,3.225497,0.450841
NA,1362,3.204736,0.420701
OC,85,3.294118,0.395617
SA,168,3.168155,0.435595


Use an __additive smoothing__ to account for AF's mean and OC's mean<br>
set $m = 100$: there must be at least 100 values for the sample mean to overtake the global mean.

In [70]:
# https://maxhalford.github.io/blog/target-encoding/
def calc_smooth_mean(df, by, on, m):
    # Compute the global mean
    mean = df[on].mean()

    # Compute the number of values and the mean of each group
    agg = df.groupby(by)[on].agg(['count', 'mean'])
    counts = agg['count']
    means = agg['mean']

    # Compute the "smoothed" means
    smooth = (counts * means + m * mean) / (counts + m)

    # Replace each value by the according smoothed mean
    return df[by].map(smooth)

In [71]:
choco_combine2['company_continent'] = calc_smooth_mean(df = choco_combine2, by = 'company_continent', 
                                                       on = 'rating', m = 100)

In [72]:
choco_combine2['company_continent']

0       3.205074
1       3.205074
2       3.205074
3       3.223531
4       3.223531
          ...   
2442    3.205074
2443    3.205074
2444    3.205074
2445    3.205074
2446    3.205074
Name: company_continent, Length: 2447, dtype: float64

### Target Encode review_date

From EDA, we conclude that the review_date is not time-correlated, so we treat the review_date (in year) as categorical variables instead of numerical variables. We also apply target encoding for this feature.

In [73]:
choco_combine3 = choco_combine2.copy()

In [74]:
choco_combine3.columns

Index(['ref', 'company', 'company_location', 'review_date',
       'country_of_bean_origin', 'specific_bean_origin_or_bar_name',
       'cocoa_percent', 'rating', 'counts_of_ingredients', 'taste',
       'cocoa_butter', 'vanilla_have_not_vanilla', 'vanilla',
       'vanilla_have_vanilla', 'lecithin', 'salt', 'sugar',
       'sweetener_without_sugar', 'nutty', 'sweet', 'cocoa', 'roasty',
       'creamy', 'earthy', 'sandy', 'fatty', 'floral', 'intense', 'spicy',
       'sour', 'molasses', 'woody', 'vanilla_taste', 'sticky', 'fruit',
       'smokey', 'company_continent', 'origin_continent'],
      dtype='object')

In [75]:
stats_Summary = choco_combine3['rating'].groupby(choco_combine3['review_date']).agg(['count', 'mean', 'std'])
stats_Summary

,count,mean,std
review_date,,,
2006,60,3.062500,0.660035
2007,69,3.166667,0.598200
2008,84,3.038690,0.494692
2009,113,3.088496,0.439308
2010,94,3.188830,0.431748
2011,153,3.271242,0.475508
2012,180,3.206944,0.464396
2013,178,3.214888,0.430760
2014,243,3.197531,0.409184


In [76]:
choco_combine3['review_date'] = choco_combine3['review_date'].map(stats_Summary['mean'])

# Split the combined set back to Train and Test

Recall that train data is the first 2224 rows and test data is from index 2224 to the end.

In [77]:
choco_combine_fin = choco_combine3.copy()

In [78]:
choco_train_fin = choco_combine_fin.loc[:2223, :]

In [79]:
choco_test_fin = choco_combine_fin.loc[2224:, :]
choco_test_fin.index = np.arange(len(choco_test_fin))

In [80]:
# Save data to file

choco_train_fin.to_csv('chocolate_train.csv')
choco_test_fin.to_csv('chocolate_test.csv')